In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import statistics
import kmax as kx
import utility as util
from hierarchy import *
from scipy.optimize import fsolve
from IPython.core.debugger import set_trace

# Simulations

## Erdos-Renyi Graphs $(G_{n,p})$

For an Erdos-Renyi graph $G\left(n,\frac{\lambda}{n}\right)$, we expect
$$
TODO
$$

For Erdos-Renyi random graph G(n, lam/n) compute the x value / n, for 1000 samples with n = 10000 or more. This should converge to E(X/N) where X =  x value for a Poisson(lam) branching process and N = number of nodes in the same Poisson(lam) branching process. X and N are correlated and we computed E(X), E(N) explicitly. What we need is E(X/N): this can be computed by simulation many Galton Watson Tree samples, for each sample compute X/N, take their average.

### Approximation of $\mathbb{E}(X/N)$

In [2]:
def X_sample_GNP(sample_tree):
    X = 0
    for nodes in nx.connected_components(sample_tree):
        subgraph = sample_tree.subgraph(nodes)
        path, _, Xi = util.path_cover(subgraph, list(nodes)[0])
        X += Xi
    return X

In [3]:
# TODO
def iterate_GNP(lam, num_samples=100, n=10000):
    Xs = 0
    Ns = 0
    for _ in range(num_samples):
        # Erdos-Reyni Graph
        sample_tree = nx.fast_gnp_random_graph(n, lam/n)

        for u,v in sample_tree.edges():
            sample_tree[u][v]['weight'] = 1

        # Calculate EX/EN for current sample
        Xs += X_sample_GNP(sample_tree)
        Ns += nx.number_of_nodes(sample_tree)
    return Xs / Ns

## GW Branching Process

For a Poisson branching process with $\lambda < 1$, we expect that
$$
\frac{X}{n}\underset{n\to\infty}{\longrightarrow}\frac{\mathbb{E}X}{\mathbb{E}N}
$$
where $X$ is the last-passage time of a GWBP tree with parameter $\lambda$

In theory,
$$
\mathbb{E}X = \frac{2-(\lambda p+2)e^{-\lambda p}}{1-\lambda}~~~~~\text{and}~~~~~\mathbb{E}N=\frac{1}{1-\lambda}
$$
where $p:=p(\lambda)$ is a solution to the equation $$pe^{\lambda p} = 1+\lambda p.$$

Thus
$$
\frac{\mathbb{E}X}{\mathbb{E}N} = 2-(\lambda p+2)e^{-\lambda p}.
$$
For example,
$
\lambda = 0.7 \implies p\approx 0.874161.
$

### Computation of $p$ + theoretical target values

In [41]:
# CONSTANTS
LAM = 0.7

In [5]:
def func(p):
    return [p[0] * np.exp(LAM*p[0]) - (1 + LAM * p[0])]

In [6]:
p = float(fsolve(func,0)[0])
EX_EN = 2-(2+LAM*p)*math.exp(-LAM*p)
EX = EX_EN / (1-LAM)

In [7]:
print(f'p = {p}')
print(f'EX/EN = {EX_EN}')
print(f'EX = {EX}')

p = 0.8741606425132245
EX/EN = 0.583526626641014
EX = 1.9450887554700462


### Approximation of $\frac{\mathbb{E}X}{\mathbb{E}N}$

We know $\mathbb{E}N=\frac{1}{1-\lambda}$ so we only need to compute $\mathbb{E}X$.

In [20]:
def iterate_GWBP_EX(num_samples=100):
    Xs = []
    for _ in range(num_samples):
        # Poisson Branching
        sample_tree = nx.Graph(GWBP(LAM, MAXLEVEL=100))
        for u,v in sample_tree.edges():
            sample_tree[u][v]['weight'] = 1

        # Get X for sample_tree
        _,_,X = util.path_cover(sample_tree, list(sample_tree.nodes())[0])
        Xs.append(X)
    return Xs

In [44]:
Xs = iterate_GWBP_EX(num_samples=10000)
EX_sampled = statistics.mean(Xs)
EX_sampled

1.9518

In [45]:
EX_EN_sampled = EX_sampled * (1 - LAM)
EX_EN_sampled

0.5855400000000001

In [42]:
#plt.hist(Xs, density=True, bins=100, range=(0,30))

$\%_{\text{error}}$ of sampled $\frac{\mathbb{E}X}{\mathbb{E}N}$ = 

In [46]:
(EX_EN_sampled - EX_EN) / EX_EN

0.01725126387658309

### Approximation of $\mathbb{E}(X/N)$
For each Galton Watson branching process look at the ratio of the X value and the size of the tree, generate multiple samples, look at their average. It will be different from expectation of X divided by expectation of N, but that should be the answer for the Erdos Renyi graph X/n

In [10]:
def iterate_GWBP_EX_N(num_samples=100):
    X_Ns = 0
    for _ in range(num_samples):
        # Poisson Branching
        sample_tree = nx.Graph(GWBP(LAM, MAXLEVEL=100))
        for u,v in sample_tree.edges():
            sample_tree[u][v]['weight'] = 1

        # Get X for sample_tree
        _,_,X = util.path_cover(sample_tree, list(sample_tree.nodes())[0])
            
        # Calculate X/N for current sample
        # GW Branching Process produces a single connected (tree) component, so N = |V|
        X_Ns += X / nx.number_of_nodes(sample_tree)
    return X_Ns / num_samples

In [11]:
EX_N_GWBP = iterate_GWBP_EX_N(num_samples=10000)
EX_N_GWBP

0.3100726156077978

In [12]:
abs(EX_N_GWBP-EX)

1.6350161398622485

In [13]:
1 - mean/EX

NameError: name 'mean' is not defined

### Graphing

In [ ]:
plt.plot(ns, values)
plt.xlabel("n")
plt.ylabel("EX")
plt.axhline(y=statistics.mean(values), color='red')